In [1]:
import sys, os
import matplotlib.pyplot as plt
import numpy as np
CURR_DIR = os.path.dirname('__file__')
ROOT_DIR=os.path.join(os.getcwd() ,'..')
sys.path.append(ROOT_DIR)

In [2]:
from omegaconf import OmegaConf
#load configs
config = OmegaConf.load(os.path.join(ROOT_DIR,'run_scripts/configs/config.yaml'))

In [3]:
config

{'model': {'target': 'models.classifier.Classifier', 'params': {'data_dir': '/home/akshay/Documents/cse_sem_7/conxai_assignment/cloned/stage-classification/data', 'image_size': 150, 'channels': 3, 'num_classes': 7, 'batch_size': 16, 'max_tsteps': 10, 'learning_rate': 0.001, 'loss_type': 'cross_entropy', 'nn_model': {'target': 'models.cnn.CNNClassification', 'params': {'input_channels': 3, 'num_classes': 7}}}}}

In [4]:
from utilities.initialize_configs import instantiate_from_configs
classifierModel = instantiate_from_configs(config.model)

2024-01-22 14:40:10.866854: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-22 14:40:10.866879: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
# %%capture
# ddpmModel.cuda()

In [6]:
for batch in classifierModel.train_dataloader():
    print(batch[1].size())
    break

torch.Size([16])


In [8]:
%%capture
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import TQDMProgressBar, Timer, ModelCheckpoint
import time
from pytorch_lightning.loggers import WandbLogger

# stop training after 12 hours
timer = Timer(duration="00:12:00:00")

# or provide a datetime.timedelta
from datetime import timedelta
timer = Timer(duration=timedelta(weeks=1))
timestr = time.strftime("%d%m%y-%H%M%S")
loss_type = classifierModel.loss_type
alpha = classifierModel.learning_rate
task = 'cnn-classification'

checkpoint = ModelCheckpoint(
#     dirpath=CKPT_DIR,
    save_top_k=2,
    monitor="val/loss_simple",    
    filename=timestr + "-{epoch}-{step}-" + "-loss=%s"%loss_type + "-a=%.1E"%alpha
)
# trainer = Trainer(max_steps=ddpmModel.max_tsteps, accelerator="gpu", devices=-1, logger=wandb_logger, callbacks=[TQDMProgressBar(refresh_rate=10), timer, checkpoint]) # To use all available GPUs put -1 or '-1'
trainer = Trainer(max_steps=classifierModel.max_tsteps, callbacks=[TQDMProgressBar(refresh_rate=10), timer, checkpoint]) # To use all available GPUs put -1 or '-1'
trainer.fit(classifierModel)

# query training/validation/test time (in seconds)
print("Total training time: %.2f mins" % (timer.time_elapsed("train")/60))

# save last checkpoint
timestr = time.strftime("%d%m%y-%H%M%S")
train_loss = "{:.4f}".format(trainer.logged_metrics['train/loss'].item())
CKPT_NAME = timestr + "_lastckpt_tloss_" + str(train_loss) + ".ckpt"
logger_version = "version_"+str(trainer.logger.version)
PATH = "lightning_logs/" + logger_version+ "/checkpoints/"+CKPT_NAME
trainer.save_checkpoint(PATH)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type              | Params
--------------------------------------------
0 | model | CNNClassification | 86.6 M
--------------------------------------------
86.6 M    Trainable params
0         Non-trainable params
86.6 M    Total params
346.361   Total estimated model params size (MB)
